In [2]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time
import configparser
import os
import pymysql

In [3]:
config = configparser.ConfigParser()
config.read("/home/yuan/Documents/config.ini")
config.sections()

['office_ssh', 'office_mysql']

In [4]:
modification_site = pd.read_csv("/home/yuan/Documents/pdb_chain_ksites.csv")

In [5]:
modification_site[0:5]

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites
0,3g8c,A,P24182,1,444,1,444,1,444,27
1,3g8c,B,P24182,1,444,1,444,1,444,27
2,3g8d,A,P24182,1,444,1,444,1,444,27
3,3g8d,B,P24182,1,444,1,444,1,444,27
4,3jzf,A,P24182,22,470,1,449,1,449,27


In [6]:
modification_site_dic = {}
for i in range(len(modification_site)):
    if modification_site.loc[i,"PDB"] not in modification_site_dic:
        modification_site_dic[modification_site.loc[i,"PDB"]] = {modification_site.loc[i,"CHAIN"]:[modification_site.loc[i,"Ksites"]]}
    elif modification_site.loc[i,"CHAIN"] not in modification_site_dic[modification_site.loc[i,"PDB"]]:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]] = [modification_site.loc[i,"Ksites"]]
    else:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]].append(modification_site.loc[i,"Ksites"])
for protein in modification_site_dic:
    for chain in modification_site_dic[protein]:
        modification_site_dic[protein][chain] = list(modification_site_dic[protein][chain])
        

In [7]:
unique_list = modification_site[["PDB","CHAIN"]].drop_duplicates()

In [8]:
unique_list = unique_list.reset_index(drop=True)

In [9]:
backbone = ["N","CA","C","O"]
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]

In [10]:
def n_nearest_neighbour_by_seq(PDB_ID,Chain,res_ID,n):
    if n >= res_ID:
        return []
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain = \"{}\" and da.ID_1= \"{}\";".format(PDB_ID,Chain,res_ID)  
                sql = sql_select+sql_where
                data = pd.read_sql_query(sql, conn)
        head_part = data.groupby(["ID"]).first().iloc[(res_ID-n-1):(res_ID-1),]
        tail_part = data.groupby(["ID"]).first().iloc[res_ID:(res_ID+n),]
        return(pd.concat([head_part,tail_part])) #from 2 to exclude self vs self

    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()

In [22]:
def One_residue_retrieval(residue_1,PDB_ID,Chain):
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain=\"{}\" and da.ID_1 = {};".format(PDB_ID,Chain,residue_1)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
                data = data_1
        
    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [11]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

from sklearn.metrics import classification_report, roc_auc_score, roc_curve, make_scorer, confusion_matrix, \
    recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, GroupKFold, cross_val_score
from sklearn.metrics import auc, precision_recall_curve,accuracy_score
from sklearn.utils import shuffle
from sklearn import tree, svm, naive_bayes, neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder,LabelEncoder
from scipy import interp
import matplotlib.pyplot as plt

In [12]:
clfs = {'decision_tree': tree.DecisionTreeClassifier(),
        'naive_gaussian': naive_bayes.GaussianNB(),
        'naive_mul': naive_bayes.MultinomialNB(),
        'K_neighbor': neighbors.KNeighborsClassifier(),
        'bagging_knn': BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5, max_features=0.5),
        'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5, max_features=0.5),
        'random_forest': RandomForestClassifier(n_estimators=200),
        'adaboost': AdaBoostClassifier(n_estimators=200),
        'gradient_boost': GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0),
        'svm': svm.SVC(C=0.9,kernel='rbf')
        }
#'svm': svm.SVC(probability=True)

In [30]:


aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(aminoAcidCodes)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)



OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=False)


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [14]:
amino_acid_property_by_res = {"ALA":1,"ILE":1,"LEU":1,"MET":1,"VAL":1,
                      "PHE":2, "TRP":2,"TYR":2,
                       "ASN":3,"CYS":3,"GLN":3,"SER":3,"THR":3,
                      "ASP":4,"GLU":4,"ARG":5,"HIS":5,"LYS":5,"GLY":6,"PRO":6}
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform([1,2,3,4,5,6])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)



OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=False)


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# After using using CD-HIT to remove redundant samples

In [15]:
cd_hit_7 = pd.read_csv("retained_CDHIT_0.7_PTM.csv")

In [16]:
cd_hit_7.head()

,pdbID,Chain
0,1ln4,A
1,1lop,A
2,1m63,A
3,1m63,B
4,1m63,D


#### 6 neighbours 1:1 = pos:neg

In [17]:
pos_sample_and_neig = pd.DataFrame()
for key, value in modification_site_dic.items():
    for chain, sites in value.items():
        if chain in list(cd_hit_7[cd_hit_7["pdbID"]==key]["Chain"]):
            for site in sites:
                neigh = n_nearest_neighbour_by_seq(key,chain,site,1)
                if len(neigh) > 0:
                    pos_sample_and_neig = pos_sample_and_neig.append(neigh.iloc[0,:])

In [18]:
pos_sample_and_neig = pos_sample_and_neig[neigh.columns]

In [19]:
pos_sample_and_neig = pos_sample_and_neig.astype({"ID_1":int,"ID_2":int})

In [20]:
pos_sample_and_neig.head()

,pdbID,chain,ID_1,Res_1,ID_2,Res_2,Distance,Angle
740417,3jzf,A,27,LYS,26,ILE,7.797483,68.099175
777277,3jzf,A,124,LYS,123,MET,8.427515,66.838669
936647,1o8c,A,50,LYS,49,GLY,7.403862,29.682107
988322,1o8c,A,209,LYS,208,GLU,8.483157,56.747731
1238230,2eck,A,13,LYS,12,GLY,5.496251,84.293718


In [23]:
neg_sample_and_neig = pd.DataFrame()
for i in range(len(pos_sample_and_neig)):
    result_step1 = One_residue_retrieval(pos_sample_and_neig.iloc[i,2],pos_sample_and_neig.iloc[i,0],pos_sample_and_neig.iloc[i,1])
    
    result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard

    result_step3 = result_step2[~result_step2.ID_2.isin(modification_site_dic[pos_sample_and_neig.iloc[i,0]][pos_sample_and_neig.iloc[i,1]])] # activate discard
    #print(result_step3)
    result_step4 = result_step3[result_step3["Res_2"]=="LYS"]
    #print(result_step4)
    #print(result_step4.sample(1,random_state = 1))
    #neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(5,replace = True)])
    #num_total_neg = len(result_step4)
    #neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(num_total_neg//3,replace = False)])
    neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(1,replace = True)])

In [24]:
pos_sample_and_neig_pruned = pos_sample_and_neig.iloc[:,[0,1,2,3]].copy(deep=True)
pos_sample_and_neig_pruned["type"] = "Pos"
pos_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [25]:
neg_sample_and_neig_pruned = neg_sample_and_neig.iloc[:,[0,1,4,5]].copy(deep=True)
neg_sample_and_neig_pruned["type"] = "Neg"
neg_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [26]:
pos_neg_sample = pd.concat([pos_sample_and_neig_pruned,neg_sample_and_neig_pruned])

In [27]:
neighbour_information_list = []
num_pos = 0
num_neg = 0
for i in range(0,len(pos_neg_sample)):
    neigh = n_nearest_neighbour_by_seq(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],12)
    if len(neigh) > 0:
        neigh = neigh.iloc[:,5:8].values.flatten()
        neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))
        if pos_neg_sample.iloc[i,4] == "Pos":
            num_pos = num_pos + 1
        else:
            num_neg = num_neg + 1

In [32]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    for element in each_res[1:]:
        if isinstance(element,str):
            one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
        else:
            one_res_data.append(element)
#         if isinstance(element,str):
#             pass
#         else:
#             one_res_data.append(element)    
    
    train_dataset.append(one_res_data)

    
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<len(train_dataset[0])):
        wrong_sample.append(i)

y_label =[1 for x in range(0,num_pos)]
y_label.extend([2 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_pos+num_neg) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_pos+num_neg) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

for clf in clfs:
    print(clf)
    scores_acc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='accuracy',n_jobs=5,cv=10)
    scores_roc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='roc_auc',n_jobs=5,cv=10)
    scores_precision = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='precision',n_jobs=5,cv=10)
    scores_recall = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='recall',n_jobs=5,cv=10)
    print('accuracy:',"{:.4f}".format(sum(scores_acc)/10),'roc:',"{:.4f}".format(sum(scores_roc)/10), \
         'precision:',"{:.4f}".format(sum(scores_precision)/10),'recall:',"{:.4f}".format(sum(scores_recall)/10))

decision_tree
accuracy: 0.5662 roc: 0.5716 precision: 0.5988 recall: 0.5738
naive_gaussian
accuracy: 0.5217 roc: 0.5341 precision: 0.5398 recall: 0.4973
naive_mul
accuracy: 0.5083 roc: 0.5077 precision: 0.5290 recall: 0.4608
K_neighbor
accuracy: 0.5178 roc: 0.5313 precision: 0.5347 recall: 0.5458
bagging_knn
accuracy: 0.5100 roc: 0.5216 precision: 0.5123 recall: 0.6006
bagging_tree
accuracy: 0.5846 roc: 0.5854 precision: 0.5668 recall: 0.6793
random_forest
accuracy: 0.5879 roc: 0.6042 precision: 0.5795 recall: 0.7288
adaboost
accuracy: 0.5550 roc: 0.5590 precision: 0.5688 recall: 0.5802
gradient_boost
accuracy: 0.5489 roc: 0.5534 precision: 0.5601 recall: 0.5920
svm
accuracy: 0.6285 roc: 0.6047 precision: 0.5821 recall: 1.0000


# Using each site in original study

In [124]:
samples_in_paper = pd.read_csv("/home/yuan/Documents/pdb_chain_ksites_huamn_pos_neg_in_paper.csv",header=0)

In [125]:
samples_in_paper.iloc[0:5]

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites,class,k_site_uniprot
0,4crw,A,A5YKK6,7,231,1093,1317,1093,1317,1227,n,1227
1,4gmj,A,A5YKK6,5,229,1093,1317,1093,1317,1227,n,1227
2,4gmj,C,A5YKK6,5,229,1093,1317,1093,1317,1227,n,1227
3,4gmj,E,A5YKK6,5,229,1093,1317,1093,1317,1227,n,1227
4,4gml,A,A5YKK6,11,235,1093,1317,1093,1317,1227,n,1227


In [160]:
sum(samples_in_paper.groupby(by=["SP_PRIMARY","k_site_uniprot","class"]).head(1)["class"]=="p")

938

In [129]:
test = samples_in_paper.groupby(by=["SP_PRIMARY","k_site_uniprot","class"]).groups

In [140]:
test[('A5YKK6',1227,'n')]

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [168]:
pos_sample_and_neig.head()

,pdbID,chain,ID_1,Res_1,ID_2,Res_2,Distance,Angle
ID,,,,,,,,
740417,3jzf,A,27,LYS,26,ILE,7.797483,68.099175
740419,3jzf,A,27,LYS,28,THR,8.190358,25.060192
777277,3jzf,A,124,LYS,123,MET,8.427515,66.838669
777279,3jzf,A,124,LYS,125,LYS,9.703662,61.074339
936647,1o8c,A,50,LYS,49,GLY,7.403862,29.682107


In [163]:
first_entry = samples_in_paper.groupby(by=["SP_PRIMARY","k_site_uniprot","class"]).head(1)

In [164]:
first_entry[first_entry["class"]=="p"]

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites,class,k_site_uniprot
10,2pkt,A,O00151,2,88,1,87,1,87,71,p,71
11,2lkn,A,O00170,1,165,2,166,2,166,112,p,112
12,4aif,A,O00170,1,144,172,315,172,315,273,p,273
14,5l4k,Q,O00231,1,422,1,422,1,422,185,p,185
34,5l4k,P,O00232,1,456,1,456,1,456,212,p,212
78,5l4k,P,O00232,1,456,1,456,1,456,368,p,368
100,5l4k,P,O00232,1,456,1,456,1,456,448,p,448
123,2rmy,A,O00499,1,33,1,33,1,33,13,p,13
125,1n99,A,O00560,6,166,113,273,113,273,124,p,124
136,1n99,A,O00560,6,166,113,273,113,273,223,p,223
